In [5]:
# ⬅︎ Cell 1: Imports & Config
import json
import os
import warnings

import joblib
import numpy as np

# Optuna
import optuna
import pandas as pd
from lightgbm import LGBMClassifier
from optuna.pruners import MedianPruner
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.impute import SimpleImputer
from sklearn.metrics import average_precision_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# מודלים
from xgboost import XGBClassifier

warnings.filterwarnings("ignore")

RANDOM_STATE = 42
DATA_PATH = "../data/raw/telco_churn.csv"  # התאם אם צריך
TARGET_COL = "Churn"


In [6]:
import pathlib
import sys

# מזהה את שורש הפרויקט כך ש-src תהיה בתוכו
cwd = pathlib.Path().resolve()
project_root = cwd if (cwd / "src").exists() else cwd.parent  # אם אתה בתוך notebooks/
sys.path.append(str(project_root))

# בדיקה מהירה:
assert (project_root / "src" / "features" / "build_features.py").exists()

from src.features.build_features import AddFeatures

print("OK:", AddFeatures)


OK: <class 'src.features.build_features.AddFeatures'>


In [7]:
# ⬅︎ Cell 2: Load & Basic cleaning
df = pd.read_csv(DATA_PATH)

# Telco quirks
# TotalCharges מגיע כמחרוזת; ננקה לרמה נומרית ונטפל בחסרים
df["TotalCharges_num"] = pd.to_numeric(df["TotalCharges"], errors="coerce")
mask_na = df["TotalCharges_num"].isna()
df.loc[mask_na, "TotalCharges_num"] = df.loc[mask_na, "MonthlyCharges"] * df.loc[mask_na, "tenure"].clip(lower=1)

# יעד לבוליאני
df[TARGET_COL] = df[TARGET_COL].map({"Yes": 1, "No": 0}).astype(int)

# פיצ’ר־אנג’ינירינג קל בתוך מחלקת טרנספורמר כדי לשלב ב-Pipeline
service_cols = [
    "PhoneService","MultipleLines","InternetService","OnlineSecurity","OnlineBackup",
    "DeviceProtection","TechSupport","StreamingTV","StreamingMovies"
]



In [8]:
# ⬅︎ Cell 3: Split
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL].values

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, stratify=y, random_state=RANDOM_STATE
)
X_valid, X_test, y_valid, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=RANDOM_STATE
)

pos = y_train.sum()
neg = len(y_train) - pos
scale_pos_weight = neg / max(pos, 1)
scale_pos_weight


2.7691131498470947

In [9]:
# ⬅︎ Cell 4: Preprocessing pipeline
feature_adder = AddFeatures(service_cols=service_cols)

numeric_proc = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler(with_mean=False))  # with_mean=False לשילוב עם sparse
])

categorical_proc = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_proc, selector(dtype_include=["int64","float64","int32","float32","uint8","bool"])),
        ("cat", categorical_proc, selector(dtype_include=object))
    ],
    remainder="drop"
)

# נבנה Pipeline כללי: add features -> preprocess -> model (יוחדר בדינמיות)
def make_pipeline(model):
    return Pipeline(steps=[
        ("add", feature_adder),
        ("prep", preprocessor),
        ("model", model)
    ])


In [10]:
# ⬅︎ Cell 5: Eval helper
def evaluate(model, X_tr, y_tr, X_va, y_va):
    proba_tr = model.predict_proba(X_tr)[:,1]
    proba_va = model.predict_proba(X_va)[:,1]
    metrics = {
        "roc_auc_train": roc_auc_score(y_tr, proba_tr),
        "pr_auc_train":  average_precision_score(y_tr, proba_tr),
        "roc_auc_valid": roc_auc_score(y_va, proba_va),
        "pr_auc_valid":  average_precision_score(y_va, proba_va),
    }
    return metrics


In [11]:
# ⬅︎ Cell 6: Optuna objective (5-fold CV on train)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

def objective(trial):
    model_type = trial.suggest_categorical("model_type", ["xgb", "lgbm"])
    
    if model_type == "xgb":
        params = {
            "n_estimators": trial.suggest_int("xgb_n_estimators", 200, 800),
            "max_depth": trial.suggest_int("xgb_max_depth", 3, 8),
            "learning_rate": trial.suggest_float("xgb_lr", 1e-2, 3e-1, log=True),
            "subsample": trial.suggest_float("xgb_subsample", 0.6, 1.0),
            "colsample_bytree": trial.suggest_float("xgb_colsample", 0.6, 1.0),
            "reg_alpha": trial.suggest_float("xgb_alpha", 1e-8, 10.0, log=True),
            "reg_lambda": trial.suggest_float("xgb_lambda", 1e-8, 10.0, log=True),
            "min_child_weight": trial.suggest_int("xgb_min_child_weight", 1, 10),
            "gamma": trial.suggest_float("xgb_gamma", 0.0, 5.0),
            "random_state": RANDOM_STATE,
            "n_jobs": -1,
            "objective": "binary:logistic",
            "tree_method": "hist",
            "eval_metric": "aucpr",
            "scale_pos_weight": scale_pos_weight
        }
        base_model = XGBClassifier(**params)
    else:
        params = {
            "n_estimators": trial.suggest_int("lgbm_n_estimators", 300, 1000),
            "num_leaves": trial.suggest_int("lgbm_num_leaves", 31, 255),
            "learning_rate": trial.suggest_float("lgbm_lr", 1e-2, 3e-1, log=True),
            "subsample": trial.suggest_float("lgbm_subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("lgbm_colsample", 0.5, 1.0),
            "min_child_samples": trial.suggest_int("lgbm_min_child_samples", 5, 100),
            "reg_alpha": trial.suggest_float("lgbm_alpha", 1e-8, 10.0, log=True),
            "reg_lambda": trial.suggest_float("lgbm_lambda", 1e-8, 10.0, log=True),
            "random_state": RANDOM_STATE,
            "n_jobs": -1,
            "objective": "binary",
            "metric": "average_precision",
            "verbose": -1,
            "scale_pos_weight": scale_pos_weight
        }
        base_model = LGBMClassifier(**params)
    
    pipe = make_pipeline(base_model)

    pr_aucs = []
    for fold, (tr_idx, va_idx) in enumerate(cv.split(X_train, y_train), start=1):
        X_tr, X_va = X_train.iloc[tr_idx], X_train.iloc[va_idx]
        y_tr, y_va = y_train[tr_idx], y_train[va_idx]

        pipe.fit(X_tr, y_tr)
        proba = pipe.predict_proba(X_va)[:, 1]
        pr_auc = average_precision_score(y_va, proba)
        pr_aucs.append(pr_auc)
        
        trial.report(np.mean(pr_aucs), step=fold)
        if trial.should_prune():
            raise optuna.TrialPruned()
    
    return float(np.mean(pr_aucs))


In [12]:
# ⬅︎ Cell 7: Run study
study = optuna.create_study(direction="maximize", pruner=MedianPruner())
study.optimize(objective, n_trials=40, timeout=None, n_jobs=1, show_progress_bar=True)

print("Best value (PR-AUC):", study.best_value)
print("Best params:", study.best_params)


[I 2025-09-04 17:00:19,340] A new study created in memory with name: no-name-01872ff7-c0a9-40f0-837b-5700b35bd15c
Best trial: 0. Best value: 0.66435:   2%|▎         | 1/40 [00:14<09:43, 14.97s/it]

[I 2025-09-04 17:00:34,312] Trial 0 finished with value: 0.6643503380345617 and parameters: {'model_type': 'xgb', 'xgb_n_estimators': 675, 'xgb_max_depth': 6, 'xgb_lr': 0.011660629980382855, 'xgb_subsample': 0.7847610420673713, 'xgb_colsample': 0.7263495806606615, 'xgb_alpha': 0.00017955931112961662, 'xgb_lambda': 0.00036364163408810776, 'xgb_min_child_weight': 6, 'xgb_gamma': 1.7451964095804362}. Best is trial 0 with value: 0.6643503380345617.


Best trial: 1. Best value: 0.667969:   5%|▌         | 2/40 [00:23<07:15, 11.45s/it]

[I 2025-09-04 17:00:43,302] Trial 1 finished with value: 0.6679689693361087 and parameters: {'model_type': 'xgb', 'xgb_n_estimators': 219, 'xgb_max_depth': 7, 'xgb_lr': 0.011614068148080407, 'xgb_subsample': 0.763941062613682, 'xgb_colsample': 0.6705027321313649, 'xgb_alpha': 3.035665372093254e-06, 'xgb_lambda': 4.671822394047869e-06, 'xgb_min_child_weight': 9, 'xgb_gamma': 1.6859117171732074}. Best is trial 1 with value: 0.6679689693361087.


Best trial: 1. Best value: 0.667969:   8%|▊         | 3/40 [00:33<06:28, 10.50s/it]

...

[I 2025-09-04 17:02:21,286] Trial 37 finished with value: 0.6697029111422438 and parameters: {'model_type': 'xgb', 'xgb_n_estimators': 514, 'xgb_max_depth': 3, 'xgb_lr': 0.010008160123367405, 'xgb_subsample': 0.8103149113392549, 'xgb_colsample': 0.7036328165381951, 'xgb_alpha': 2.629847586052216e-05, 'xgb_lambda': 0.0016445122994628713, 'xgb_min_child_weight': 6, 'xgb_gamma': 3.9398614047757765}. Best is trial 3 with value: 0.6721159252417129.


Best trial: 3. Best value: 0.672116:  98%|█████████▊| 39/40 [02:03<00:02,  2.46s/it]

[I 2025-09-04 17:02:22,420] Trial 38 pruned. 


Best trial: 3. Best value: 0.672116: 100%|██████████| 40/40 [02:04<00:00,  3.12s/it]

[I 2025-09-04 17:02:23,952] Trial 39 pruned. 
Best value (PR-AUC): 0.6721159252417129
Best params: {'model_type': 'xgb', 'xgb_n_estimators': 679, 'xgb_max_depth': 4, 'xgb_lr': 0.06139407172509567, 'xgb_subsample': 0.8496706063048791, 'xgb_colsample': 0.7857466464862991, 'xgb_alpha': 3.223882195537856e-05, 'xgb_lambda': 9.984111524261742, 'xgb_min_child_weight': 3, 'xgb_gamma': 4.398403650674861}


In [13]:
# ⬅︎ Cell 8: Refit on train+valid and evaluate on test
best = study.best_params
best_type = best["model_type"]

if best_type == "xgb":
    model = XGBClassifier(
        n_estimators=best["xgb_n_estimators"],
        max_depth=best["xgb_max_depth"],
        learning_rate=best["xgb_lr"],
        subsample=best["xgb_subsample"],
        colsample_bytree=best["xgb_colsample"],
        reg_alpha=best["xgb_alpha"],
        reg_lambda=best["xgb_lambda"],
        min_child_weight=best["xgb_min_child_weight"],
        gamma=best["xgb_gamma"],
        random_state=RANDOM_STATE,
        n_jobs=-1,
        objective="binary:logistic",
        tree_method="hist",
        eval_metric="aucpr",
        scale_pos_weight=scale_pos_weight
    )
else:
    model = LGBMClassifier(
        n_estimators=best["lgbm_n_estimators"],
        num_leaves=best["lgbm_num_leaves"],
        learning_rate=best["lgbm_lr"],
        subsample=best["lgbm_subsample"],
        colsample_bytree=best["lgbm_colsample"],
        min_child_samples=best["lgbm_min_child_samples"],
        reg_alpha=best["lgbm_alpha"],
        reg_lambda=best["lgbm_lambda"],
        random_state=RANDOM_STATE,
        n_jobs=-1,
        objective="binary",
        metric="average_precision",
        verbose=-1,
        scale_pos_weight=scale_pos_weight
    )

pipe_best = make_pipeline(model)
pipe_best.fit(pd.concat([X_train, X_valid]), np.concatenate([y_train, y_valid]))

# Evaluate
metrics = evaluate(pipe_best, pd.concat([X_train, X_valid]), np.concatenate([y_train, y_valid]), X_test, y_test)
metrics


{'roc_auc_train': 0.8822909256898314,
 'pr_auc_train': 0.7224447789715509,
 'roc_auc_valid': 0.8423042704626335,
 'pr_auc_valid': 0.6646421326089454}

In [14]:
# ⬅︎ Cell 9: Save artifacts
os.makedirs("../models", exist_ok=True)

joblib.dump(pipe_best, "../models/best_pipeline.pkl")

# שמירת מטא־דאטה כולל שמות פיצ'רים אחרי OneHot (יעיל להסברים וחשיבות פיצ'רים)
# שים לב: get_feature_names_out קיים בגרסאות sklearn חדשות
prep = pipe_best.named_steps["prep"]
feature_names = prep.get_feature_names_out()
meta = {
    "model_type": best_type,
    "best_params": best,
    "feature_names": feature_names.tolist()
}
with open("../models/metadata.json", "w") as f:
    json.dump(meta, f, indent=2)

print("Saved ../models/best_pipeline.pkl and ../models/metadata.json")


Saved ../models/best_pipeline.pkl and ../models/metadata.json


In [15]:
# ⬅︎ Cell 10: Feature importance (top 25)
import numpy as np


def get_importances(pipe):
    mdl = pipe.named_steps["model"]
    if hasattr(mdl, "feature_importances_"):
        imp = mdl.feature_importances_
    elif hasattr(mdl, "booster_") and hasattr(mdl.booster_, "feature_importances_"):
        imp = mdl.booster_.feature_importances_
    else:
        return None
    names = pipe.named_steps["prep"].get_feature_names_out()
    return pd.DataFrame({"feature": names, "importance": imp}).sort_values("importance", ascending=False)

fi = get_importances(pipe_best)
if fi is not None:
    display(fi.head(25))
    fi.to_csv("../models/feature_importance_top.csv", index=False)
else:
    print("No feature_importances_ available for this model.")


,feature,importance
6032,cat__Contract_Month-to-month,0.375308
11,num__contract_term,0.182006
6012,cat__InternetService_Fiber optic,0.046414
8,num__has_fiber,0.043145
6014,cat__OnlineSecurity_No,0.037685
6034,cat__Contract_Two year,0.020311
6023,cat__TechSupport_No,0.020236
4,num__tenure_years,0.017728
6011,cat__InternetService_DSL,0.014983
12,num__monthly_x_term,0.014420


In [ ]:
# ⬅︎ Cell 11: Quick smoke test on a small sample
Xs = X_train.sample(200, random_state=RANDOM_STATE)
ys = y_train[np.isin(X_train.index, Xs.index)]

pipe_smoke = make_pipeline(LGBMClassifier(n_estimators=50, random_state=RANDOM_STATE))
pipe_smoke.fit(Xs, ys)
proba = pipe_smoke.predict_proba(Xs)[:,1]
print("Smoke PR-AUC:", average_precision_score(ys, proba))


Smoke PR-AUC: 0.9691152348567608


: 